The goal of this project is to better understand walking and impaired walking (slow, shuffling or freezing of gait), in Parkinson's disease (PD) as it compares to that of age-matched healthy controls. We use data from wearable inertial measurement unit sensors (IMUs, made by APDM Inc.) to calculate gait parameters that characterize the dynamics of people walking in both a turning and barrier course designed to elicit freezing of gait (Syrkin-Nikolau et al., Neurobiology of disease, 2017 and O'Day et al., BioRxiv, 2019 https://doi.org/10.1101/671479) and over forty meters of straight forward walking, the clinical test of Parkinsonian gait. Inertial measurement unit sensors were applied laterally to both shanks for all subjects. Methods are extensively described in the above two references. The records in this database are from patients with Parkinson's disease who experience severe walking symptoms like freezing of gait (FOG), also identified as (freezers); as well as data from people with Parkinson's disease who do not experience FOG (non-freezers). All subjects are off dopaminergic medication or off subthalamic deep brain stimulation (DBS), if applicable, for the walking trials unless specified otherwise (there's a subset of the cohort that is on DBS for some trials, see more below). We also have a small cohort of age-matched healthy controls who do not have Parkinson's disease and who completed the walking tasks. There were 20 total subjects with Parkinson's disease that completed the walking tasks, but 11 of these subjects returned for a later research visit where they completed the same tasks (at least 1 year later). This makes a total of 31 records from subjects with Parkinson's disease and 9 records from age-matched healthy control subjects, for a total of 40 records. Deep brain stimulation cohort There was a subset of the cohort with Parkinson's disease that had been treated with at least 21 months of optimized, continuous high frequency subthalmic DBS. Kinematic recordings were obtained, off medication, during randomized presentations of no, 60 Hz and 140 Hz subthalamic DBS while subjects performed the TBC. The gait data for these trials can be found in subjects with DBS folders. Subject folders Each subject folder is labeled with the subject ID. As stated above, there are 11 subjects with Parkinson's disease that have a repeated research visit where they did both the FW and TBC tasks a second time. If wanting to look at repeated subjects, then look within the group with Parkinson's disease (PD in metadata) and at last 2 digits of the subject ID, though the subjects have different prefixes, they will have the same last 2 digits if they are the same subject. This is only true in the PD cohort (for instance there is a control subject that has 20 as their last two digits, but this subject is distinct from the PD subject that has 20 as their last two digits). Metadata The metadata.yaml contains attributes including the subject's group (PD or Control), age (rounded down to nearest year), gender (F or M), dbs (yes if they have deep brain stimulation or no if not, which is relevant to the sub-cohort explained above), freezer (yes/no), FOGQ score (which is the International Freezing of Gait Questionnaire Total Score, constructed by Giladi et al., 2000 Parkinsonism and Rel Dis and has a range of 0-24), and specifically the FOGQ Item 3 score which assesses severity of freezing with the question: Do you feel that your feet get glued to the floor while walking, turning or when trying to initiate walking? The scores were as follows: 0 – never, 1 – about once a month, 2 – about once a week, 3 – about once a day, 4 – whenever walking. Freeze files Within the TBC and FW files, there are freezes.mat files and these can be loaded into Matlab and have two columns delineatning the start and end timings of freezing events, respectively, as determined with video inspection by an expert neurologist. Gaitdata Within the subject folder are two subfolders, TBC and FW. These folders contain multiple .csv files of the gait parameters that were calculated from shank IMU signals for each trial from this subject. The TBC folder contains this subject's records while walking in the turning and barrier course, and FW contains this subject's records during forty meters of straight walking up and down a 10 meter hallway (for a total of 4 hallway lengths). The TBC task has 2 gait .csv files (1 where the subject walks counterclockwise in the course, and 1 where the subject walks clockwise). 

The FW task has 4 gait .csv files (1 for each 10 meter straightaway, for a total of 40 meters). 
The .csv file* is structured as follows: 
* Column 1: Patient ID number (integer) 
* Column 2: Number of total left steps for this trial (integer) 
* Column 3: Times of peak angular velocity on left shank (sec) 
* Column 4: Left gait cycle (stride time) durations (sec) 
* Column 5: Left swing time durations (sec) 
* Column 6: Left swing angular ranges (deg) 
* Column 7: Peak shank angular velocity (deg/s) 
* Column 8: Freeze index (accelerometer)** 
* Column 9: Left leg identifier = 0 
* Column 10: Number of total right steps for this trial (integer) 
* Column 11: Times of peak angular velocity on right shank (sec) 
* Column 12: Right gait cycle (stride time) durations (sec) 
* Column 13: Right swing time durations (sec) 
* Column 14: Right swing angular ranges (deg) 
* Column 15: Peak shank angular velocity (deg/s) 
* Column 16: Freeze index (accelerometer)** 
* Column 17: Right leg identifier = 1 *any extra -1's that you see in the .csv file are just placeholders, and can be ignored ** Further information on how and the other gait parameters are calculated here (O'Day et al., BioRxiv, 2019 https://doi.org/10.1101/671479)
doi:10.18735/454q-tx50

In [2]:
import torch
import pandas as pd
import numpy as np
from torch import nn
import os
import sklearn
import matplotlib.pyplot as plt
import sys
import re
from matplotlib.pyplot import figure

In [17]:
folder_path = '/home/jm/vscode/Python/pytorch/Fall Dectection/uParkinsons'

fw_df = pd.DataFrame()
tbc_df = pd.DataFrame()

print(os.listdir(folder_path))

for folder in os.listdir(folder_path):
    for subfolder in os.listdir(folder_path+'/'+folder):
        print(subfolder, '\n')
        for fw_file in os.listdir(folder_path+'/'+folder+'/FW'):
            if fw_file.endswith(".csv"):
                print(fw_file)
                fw_csv = pd.read_csv(folder_path+'/'+folder+'/FW/'+fw_file)
                fw_df = pd.concat([fw_df, fw_csv])

['Subject11115', 'Subject10017', 'Subject11120', 'Subject10034', 'Subject11106', 'Subject10002', 'Subject10020', 'Subject11105', 'Subject10022', 'Subject11119', 'Subject11110', 'Subject10016', 'Subject10024', 'Subject11016', 'Subject11019', 'Subject10030', 'Subject11008', 'Subject10012', 'Subject10035', 'Subject11018', 'Subject11002', 'Subject10023', 'Subject10001', 'Subject11020', 'Subject11116', 'Subject11111', 'Subject11005', 'Subject10006', 'Subject10019', 'Subject11117', 'Subject11118', 'Subject11004', 'Subject11103', 'Subject11114', 'Subject11011', 'Subject11010', 'Subject11003', 'Subject10021', 'Subject10037']
metadata.yaml 

PA15_FW.h5_GaitDataV2_T1_199.2812-209.7188s.csv
PA15_FW.h5_GaitDataV2_T2_211.9375-222.3828s.csv
PA15_FW.h5_GaitDataV2_T3_226.2578-235.8203s.csv
PA15_FW.h5_GaitDataV2_T4_238.2344-248.5703s.csv
TBC 

PA15_FW.h5_GaitDataV2_T1_199.2812-209.7188s.csv
PA15_FW.h5_GaitDataV2_T2_211.9375-222.3828s.csv
PA15_FW.h5_GaitDataV2_T3_226.2578-235.8203s.csv
PA15_FW.h5_GaitDa

In [18]:
fw_df

,15,10,199.64,-1,-1.1,-1.2,-1.3,-1.4,-1.5,9,...,49.391,48.922,121.56,7.2,122.11,113.91,130.68,130.53,140.84,141.38
0,15.0,-1.0,200.72,1.07810,0.27344,65.752,401.44,1.33820,0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15.0,-1.0,201.80,1.08590,0.28125,66.519,391.65,1.58750,0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15.0,-1.0,202.85,1.04690,0.33594,63.967,283.55,1.70320,0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15.0,-1.0,204.02,1.17190,0.28125,65.069,376.01,1.96790,0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15.0,-1.0,205.08,1.05470,0.27344,65.270,382.21,1.77090,0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,NaN,NaN,NaN,0.99219,0.28906,74.685,395.72,0.81630,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142.87,143.37
2,NaN,NaN,NaN,0.97656,0.29688,77.823,400.25,0.89645,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143.84,144.37
3,NaN,NaN,NaN,1.01560,0.28906,75.660,389.25,0.92509,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.86,145.39
4,NaN,NaN,NaN,1.00000,0.28906,76.196,397.87,1.01650,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.86,146.38
